In [2]:
import xgboost

/usr/local/lib/python3.4/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
from gensim.models.keyedvectors import KeyedVectors

In [6]:
word_vectors = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)  # C binary format

In [85]:
def question2vec_advanced(questions, embeddings):
    """
    Computes question embeddings by averaging word embeddings.

    Args:
      questions (list of strings): List of questions to be embedded.
      embeddings (gensim object): Pre-trained word embeddings.

    Returns:
      ndarray of shape [num_questions, embed_size] with question embeddings.
    """
    to_return = []
    for question in (questions):
        to_mean = []
        for word in question.split():
            try:
                while word not in embeddings and len(word) > 0:
                    word = word[:-1]
                embed = embeddings[word]
                to_mean.append(embed)
            except:
#                 embed = np.zeros(embeddings.vector_size)
#                 to_mean.append(embed)
                pass
        if len(to_mean) == 0:
            to_mean = [np.zeros(embeddings.vector_size)]

        to_return.append(np.mean(to_mean, axis=0))
    return np.array(to_return)

In [10]:
123

123

In [65]:
with open('/notebooks/ucheba/Documents.csv', 'r') as f:
    docs = f.read()

In [66]:
import re

In [75]:
splitted_docs = re.split('\n.Id \d+', docs)

In [76]:
123123

123123

In [77]:
len(splitted_docs)

1400

In [89]:
def extract_text(doc):
    start_pos = 2 + doc.find('.W')
    text = doc[start_pos:].replace('\n', ' ')
    return text

In [92]:
texts = list(map(extract_text, splitted_docs))

In [93]:
len(texts)

1400

In [103]:
texts[10]

' similar solutions in compressible laminar free mixing problems . there are in supersonic aerodynamics many situations of practical interest wherein streams of different velocities and, in general, different stagnation pressures mix with one another . in the majority of these problems the interaction between the two streams takes place in the presence of an axial pressure gradient .  its effect on the characteristics of the mixing may influence significantly the performances of the devices wherein the phenomena cited above occur .  a theoretical and experimental program of research to study mixing in the presence of axial pressure gradients is being carried on at the polytechnic institute of brooklyn .'

In [88]:
splitted_docs[-1][143:]

'.W\nthe buckling shear stress of simply-supported infinitely\nlong plates with transverse stiffeners .\n  this report is an extension of previous theoretical\ninvestigations of the elastic buckling in shear of flat\nplates reinforced by transverse stiffeners . the plates are treated\nas infinitely long and simply-supported along the long\nsides . stiffeners are spaced at regular intervals, dividing the plate\ninto a number of panels of uniform size . the effect\nob bending and torsional stiffnesses of the stiffener upon the buckling\nshear stress is calculated for the complete range\nof stiffnesses, for panels with ratios of width to stiffener spacing of\ngraphical forms .'

In [87]:
splitted_docs[-1].find(".W")

143

In [84]:
splitted_docs[3].split('\n')

['',
 '.T',
 'approximate solutions of the incompressible laminar',
 'boundary layer equations for a plate in shear flow .',
 '.A',
 'yen,k.t.',
 '.B',
 'j. ae. scs. 22, 1955, 728.',
 '.W',
 'approximate solutions of the incompressible laminar',
 'boundary layer equations for a plate in shear flow .',
 '  the two-dimensional steady boundary-layer',
 'problem for a flat plate in a',
 'shear flow of incompressible fluid is considered .',
 'solutions for the boundary-',
 'layer thickness, skin friction, and the velocity',
 'distribution in the boundary',
 'layer are obtained by the karman-pohlhausen',
 'technique .  comparison with',
 'the boundary layer of a uniform flow has also',
 'been made to show the effect of',
 'vorticity .']

In [34]:
len(splitted_docs)

1482

In [24]:
splitted_docs[50]

' 48\n.T\nsupersonic flow at the surface of a circular cone at\nangle of attack .\n.A\nwillett,j.e.\n.B\nj. ae. scs. 27, 1960, 907.\n.W\nsupersonic flow at the surface of a circular cone at\nangle of attack .\n  formulas for the inviscid flow properties on the surface of a\ncone at angle of attack are derived for use in conjunction with\nthe m.i.t. cone tables .  these formulas are based upon an\nentropy distribution on the cone surface which is uniform and\nequal to that of the shocked fluid in the windward meridian\nplane .  they predict values for the flow variables which may\ndiffer significantly from the corresponding values obtained\ndirectly from the cone tables .  the differences in the magnitudes\nof the flow variables computed by the two methods tend to\nincrease with increasing free-stream mach number, cone angle\nand angle of attack .\n'

In [14]:
len(splitted_docs)

1401

In [105]:
import numpy as np

In [106]:
docs_embedded = question2vec_advanced(splitted_docs, word_vectors)

In [107]:
len(docs_embedded)

1400

In [119]:
123

123

In [120]:
import pandas as pd

In [110]:
with open('./ucheba/queries.csv', 'r') as f:
    queries = f.read().splitlines()[1:]

In [123]:
queries = pd.read_csv('./ucheba/queries.csv', sep='\t', index_col=False)

In [124]:
queries.head()

,QueryId,Query
0,1.0,what similarity laws must be obeyed when const...
1,2.0,what are the structural and aeroelastic proble...
2,4.0,what problems of heat conduction in composite ...
3,8.0,can a criterion be developed to show empirical...
4,9.0,what chemical kinetic system is applicable to ...


In [182]:
train_data = pd.read_csv("/notebooks/ucheba/relevance_train.csv", sep="\s+|\t+|\s+\t+|\t+\s+")

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [183]:
train_data.head()

,QueryId,DocumentId,Relevance
0,1,184,2
1,1,29,2
2,1,31,2
3,1,12,3
4,1,51,3


In [184]:
from collections import defaultdict

In [206]:
docs_ratings = defaultdict(int)
for elem in train_data[['DocumentId', 'Relevance']].values:
    if elem[1] == -1:
        continue
    docs_ratings[elem[0]] += elem[1]

In [207]:
len(queries)

225

In [130]:
queries.drop(225, axis=0, inplace=True)

In [134]:
queries.tail()

,QueryId,Query
220,353,papers applicable to this problem (calculation...
221,355,has anyone investigated the shear buckling of ...
222,356,papers on shear buckling of unstiffened rectan...
223,360,"in practice, how close to reality are the assu..."
224,365,what design factors can be used to control lif...


In [132]:
queries['QueryId'] = queries['QueryId'].astype(int)

In [133]:
from tqdm import tqdm

In [139]:
query_to_embed = {}
for idd, quer in tqdm(queries.values):
    query_to_embed[idd] = question2vec_advanced([quer], word_vectors)[0]

100%|██████████| 225/225 [00:00<00:00, 8500.05it/s]


In [140]:
query_to_embed[min(query_to_embed)]

array([  3.85579430e-02,   6.68431595e-02,   1.34025067e-01,
         6.53686523e-02,  -4.89908867e-02,  -4.85229492e-02,
         3.24503593e-02,  -4.71842438e-02,   7.09065720e-02,
         9.90109742e-02,   3.07189934e-02,  -1.28312171e-01,
        -6.21419288e-02,   2.63061514e-03,  -1.29801437e-01,
         4.96358238e-02,  -1.61946611e-03,   5.14404289e-02,
        -5.79956062e-02,  -8.82812515e-02,  -2.10367832e-02,
        -1.51163735e-03,  -1.91243496e-02,   5.07695526e-02,
         3.88387032e-02,  -4.12433632e-02,  -1.18787892e-01,
         7.02270493e-02,   4.16361503e-02,  -5.93587235e-02,
        -3.13161202e-02,  -6.97753876e-02,  -1.77042652e-02,
         9.18985996e-03,   3.40199806e-02,  -5.46020493e-02,
         3.55794281e-02,   1.72932949e-02,   6.48081452e-02,
         6.35681152e-02,   9.24194306e-02,   8.46354174e-04,
         1.00972496e-01,  -5.83902991e-04,  -3.26497406e-02,
        -8.61490890e-02,  -1.64347328e-02,   4.13312279e-02,
        -8.54034442e-03,

In [141]:
from scipy.linalg import norm

In [208]:
to_send = pd.read_csv('/notebooks/ucheba/relevance_test.csv')


In [188]:
to_send

,QueryId,DocumentId
0,126,974
1,126,1326
2,126,187
3,126,969
4,126,970
5,126,971
6,126,972
7,126,973
8,126,942
9,127,101


In [146]:
query_to_embed['127']

KeyError: '127'

In [153]:
query_to_embed.keys()

dict_keys([1, 2, 4, 8, 9, 10, 12, 13, 15, 18, 22, 23, 26, 27, 29, 31, 32, 33, 34, 35, 39, 40, 41, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 66, 67, 68, 69, 71, 72, 74, 79, 80, 81, 82, 83, 84, 85, 86, 87, 93, 94, 95, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 116, 118, 119, 120, 121, 122, 123, 126, 128, 130, 131, 132, 133, 135, 136, 137, 138, 139, 140, 141, 142, 143, 145, 146, 147, 148, 149, 150, 152, 153, 154, 155, 156, 157, 158, 160, 161, 163, 164, 165, 167, 168, 169, 170, 171, 173, 175, 176, 177, 181, 182, 183, 184, 187, 189, 190, 196, 200, 201, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 223, 224, 225, 226, 227, 230, 231, 232, 233, 234, 241, 245, 246, 247, 250, 251, 252, 253, 254, 255, 257, 259, 261, 264, 265, 266, 267, 268, 269, 272, 273, 274, 275, 277, 283, 284, 285, 288, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 303, 304, 306, 314, 315, 316, 317, 321, 323, 327, 331, 332, 33

In [166]:
import numpy as np

In [169]:
np.random.rand()

0.38941566065350786

In [209]:
ratings = []
c = 0
for queryid, docid in tqdm(to_send.values):
    try:
        cur_norm = norm(query_to_embed[queryid] - docs_embedded[docid - 1], 2)
        ratings.append(cur_norm)
    except:
        if docid not in docs_ratings:
            ratings.append(1000. + np.random.rand())
        else:
            ratings.append(-docs_ratings[docid])
        c += 1

100%|██████████| 847/847 [00:00<00:00, 34363.91it/s]


In [149]:
c

238

In [154]:
len(ratings)

0

In [210]:
to_send['relevance'] = ratings

In [159]:
with open('ucheba/relevance_train.csv', 'r') as f:
    nqew = f.read()

In [160]:
nqew.find('\t127\t')

-1

In [178]:
to_send['rel2'] = np.random.rand(len(to_send))

In [211]:
to_send

,QueryId,DocumentId,relevance
0,126,974,0.772774
1,126,1326,0.780790
2,126,187,0.794849
3,126,969,0.764251
4,126,970,0.789947
5,126,971,0.817192
6,126,972,0.787602
7,126,973,0.872448
8,126,942,0.791183
9,127,101,-16.000000


In [205]:
1237512 in docs_ratings

False

In [212]:
srted = to_send.sort_values(by=['QueryId', 'relevance'])

In [214]:
srted[['QueryId', 'DocumentId']].to_csv('to_send_res_mew.csv', index=None)